This code should analyze data using a Fixed Effects method.

Author: Simon Kirschberger
Partial Source:   Lost Stats. (n.d.). Fixed Effects in Linear Regression. LOST. Retrieved June 21, 2022, from https://lost-stats.github.io/Model_Estimation/OLS/fixed_effects_in_linear_regression.html


In [ ]:
#import relevant libraries
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.api as sm
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
import seaborn as sns


In [184]:
#import data
df_panel= pd.read_excel(r"C:\Users\Simon\Downloads\df_altman7.xlsx")

In [185]:
#Set filters
Altman_Lower_Bound = -30
Altman_Upper_Bound = 30
Customer_Minimum = 2
Undisclosed_Lower_Bound = 0.01
Undisclosed_Upper_Bound = 1
SCF_Lower_Bound = 0
SCF_Upper_Bound = 1
Year_Lower_Bound = 2017
Year_Upper_Bound = 2021
#Industry = ''
Country = ['Canada', 'France', 'United States of America', 'Germany', 'China',
       'Croatia', 'South Africa', 'United Kingdom', 'Japan', 'Taiwan',
       'Denmark', 'Cyprus', 'Netherlands', 'Austria', 'Switzerland',
       'Spain', 'Belgium', 'Sweden', 'Ireland; Republic of',
       'New Zealand', 'India', 'Peru', 'Israel', 'Russia', 'Tunisia',
       'Norway', 'Mexico', 'Bermuda', 'Australia', 'Brazil',
       'United Arab Emirates', 'Luxembourg', 'Jordan', 'Uruguay',
       'Poland', 'Singapore', 'Malaysia', 'Finland', 'Hong Kong',
       'Cayman Islands', 'Mauritius', 'Italy', 'Turkey', 'Chile',
       'Panama', 'Monaco', 'Korea; Republic (S. Korea)', 'Greece',
       'Sri Lanka', 'Vietnam', 'Colombia', 'Saudi Arabia', 'Indonesia',
       'Philippines', 'Jersey', 'Romania', 'Thailand', 'Pakistan',
       'Hungary', 'Egypt', 'Malta', 'Papua New Guinea', 'Qatar',
       'Kazakhstan', 'Nigeria', 'Kenya', 'Bahamas', 'Ukraine',
       'Argentina']

In [187]:
#Apply Filters
df_panel = df_panel[df_panel['Altman_Z'].between(Altman_Lower_Bound, Altman_Upper_Bound)]
df_panel = df_panel[df_panel['%_have_undisclosed_scf'].between(Undisclosed_Lower_Bound, Undisclosed_Upper_Bound)]
df_panel = df_panel[df_panel['Year'].between(Year_Lower_Bound, Year_Upper_Bound)]
df_panel = df_panel[df_panel['%_have_scf'].between(SCF_Lower_Bound, SCF_Upper_Bound)]
df_panel = df_panel[df_panel['new_customers'] >Customer_Minimum]
#df_panel = df_panel[df_panel['Industry'] ==Industry]
df_panel = df_panel[df_panel['Country'].isin(Country)]

In [188]:
#remove unnessecary columns
df_panel['percentage_have_undisclosed_scf']=df_panel['%_have_undisclosed_scf']
df_panel['percentage_have_scf']=df_panel['%_have_scf']
df_panel.drop(['Unnamed: 0', 'Unnamed: 0.1', 'new_customers', 'have_scf', 'disclose_scf', 'Year_Eikon_Helper', 'Z_is_error','Supplier_ID','%_have_undisclosed_scf','%_have_scf'] , axis=1, inplace=True)


In [189]:
#Set multiindex. CHNGE THIS TO CHANGE THE FIXED EFFECTS
data = df_panel.set_index(['Country', 'Year'])

In [192]:
#inspect df
data

Supplier_Name   Altman_Z  \
Country                  Year                                
France                   2021          2Crsi SA   3.186334   
Germany                  2021            4Sc AG   3.149493   
United States of America 2020          AAR Corp   5.434088   
                         2021          AAR Corp   7.061378   
                         2018  ACCO Brands Corp   1.066228   
...                                         ...        ...   
                         2017         iPass Inc -10.280849   
Germany                  2018   thyssenkrupp AG   0.775497   
                         2019   thyssenkrupp AG   0.222124   
                         2020   thyssenkrupp AG   2.540739   
                         2021   thyssenkrupp AG   2.803712   

                                                                Industry  \
Country                  Year                                              
France                   2021  Computers, Phones & Household Electronics   
Germany                  2021           Biotechnology & Medical Research   
United States of America 2020                        Aerospace & Defense   
                         2021                        Aerospace & Defense   
                         2018         Professional & Commercial Services   
...                                                                  ...   
                         2017                Telecommunications Services   
Germany                  2018                            Metals & Mining   
                         2019                            Metals & Mining   
                         2020                            Metals & Mining   
                         2021                            Metals & Mining   

                               percentage_have_undisclosed_scf  \
Country                  Year                                    
France                   2021                         0.250000   
Germany                  2021                         0.333333   
United States of America 2020                         0.071429   
                         2021                         0.066667   
                         2018                         0.250000   
...                                                        ...   
                         2017                         0.090909   
Germany                  2018                         0.166667   
                         2019                         0.142857   
                         2020                         0.142857   
                         2021                         0.125000   

                               percentage_have_scf  
Country                  Year                       
France                   2021             0.250000  
Germany                  2021             0.333333  
United States of America 2020             0.142857  
                         2021             0.133333  
                         2018             0.250000  
...                                            ...  
                         2017             0.090909  
Germany                  2018             0.166667  
                         2019             0.285714  
                         2020             0.285714  
                         2021             0.250000  

[5440 rows x 5 columns]

In [181]:
#Set up FE model
FE = PanelOLS(data.Altman_Z, data['percentage_have_scf'],
              entity_effects = True,
              time_effects=True,
              )

In [193]:
#Configure model
result = FE.fit(cov_type = 'clustered',
             cluster_entity=True,
              cluster_time=True
             )

In [196]:
#Print Results
print(result)
result.params

                          PanelOLS Estimation Summary                           
Dep. Variable:               Altman_Z   R-squared:                        0.0011
Estimator:                   PanelOLS   R-squared (Between):              0.1132
No. Observations:                5440   R-squared (Within):               0.0011
Date:                Tue, Jun 21 2022   R-squared (Overall):              0.0417
Time:                        17:36:26   Log-likelihood                 -1.84e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      5.7799
Entities:                          51   P-value                           0.0162
Avg Obs:                       106.67   Distribution:                  F(1,5384)
Min Obs:                       1.0000                                           
Max Obs:                       2955.0   F-statistic (robust):             3.1806
                            

percentage_have_scf    1.29326
Name: parameter, dtype: float64